Subsets II
===

***
Problem
---
Given a collection of integers that might contain duplicates, return all possible subsets.
Note: The solution must not contain duplicate subsets.

For example, if nums = [1, 2, 2], a solution is:
[[2], [1], [1, 2, 2], [2, 2], [1, 2], []]

In [44]:
import itertools
from collections.abc import Iterable

**First solution**

In [27]:
def decart(A, B):
    if not A:
        return B
    if not B:
        return A
    return [x + y for x in A for y in B]

def _subsets(collection):
    it = iter(collection)
    try:
        first = next(it)
    except StopIteration:
        return [tuple()]
    if isinstance(first, Iterable):
        first = list(first)
        first = [tuple(first[i:]) for i in range(len(first))]
    else:
        first = [(first,)]
    without = _subsets(it)
    first.append(tuple())
    return decart(first, without)


def subsets(collection):
    
    def subsets_unique(collection):
        it = iter(collection)
        try:
            first = next(it)
        except StopIteration:
            return [tuple()]
        without = subsets_unique(it)
        return decart([(first,), tuple()], without)
    
    def subsets_duplicates(collection):
        it = iter(collection)
        try:
            first = next(it)
        except StopIteration:
            return [tuple()]
        first_subsets = [tuple(first[i:]) for i in range(len(first) + 1)]
        rest = subsets_duplicates(it)
        return decart(first_subsets, rest)

    elements = sorted(collection)
    duplicates = []
    unique = []
    i, N = 0, len(elements)
    while i < N:
        j = i + 1
        while j < N and elements[j] is elements[i]:
            j += 1
        if j > i + 1:
            duplicates.append(elements[i:j])
        else:
            unique.append(elements[i])
        i = j
    
    return decart(subsets_unique(unique),
                  subsets_duplicates(duplicates))
            

**Clean**

In [5]:
def decart(A, B):
    if not A:
        return B
    if not B:
        return A
    return [x + y for x in A for y in B]


def group(collection):
    it = iter(collection)
    G = []
    try:
        lastseen = next(it)
    except StopIteration:
        pass
    else:
        while True:
            G = [lastseen]
            try:
                current = next(it)
            except StopIteration:
                break
            while current == lastseen:
                G.append(current)
                try:
                    current = next(it)
                except StopIteration:
                    break
            yield G
            lastseen = current
    yield G


def subsets(collection):
    def _subsets(collection):
        if not collection:
            return [tuple()]
        first = collection[0]
        s1 = [tuple(first[i:]) for i in range(len(first) + 1)]
        s2 = _subsets(collection[1:])
        return decart(s1, s2)
    
    return _subsets(list(group(sorted(collection))))


**Pythonic**

In [57]:
import itertools
from operator import add
from functools import reduce, partial

def py_subsets(collection):
    def subsets(elements):
        if not elements:
            return [tuple()]
        first = elements[0]
        s1 = [tuple(first[i:]) for i in range(len(first) + 1)]
        s2 = subsets(elements[1:])
        return list(map(partial(reduce, add), itertools.product(s1, s2)))
    
    s = list(map(lambda x: list(x[1]), itertools.groupby(sorted(collection))))
    return subsets(s)


In [61]:
py_subsets([1, 2, 2])

[(1, 2, 2), (1, 2), (1,), (2, 2), (2,), ()]